### imports and globals

In [ ]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
import pandas as pd

In [ ]:
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI

from enum import Enum, auto
from strenum import LowercaseStrEnum

from langchain.prompts import PromptTemplate
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.output_parsers import JsonOutputParser


# rfrom langchain_google_genai.chat_models import ChatGoogleGenerativeAI


In [ ]:
load_dotenv()

In [ ]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
OPEN_AI_KEY = os.environ["OPEN_AI_KEY"]

### prepare promt template


In [ ]:
# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    columns = ['Meal ID',
               'Date',
               'Time',
               'Meal component',
               'Quantity per serving',
               'Carbohydrates',
               'Protein',
               'Fat',
               'Calories'],
    data = [['1',
            '2025-02-25',
            '12:00',
            'Brown Bread (2 slices)',
            '2 slices',
            '20',
            '5',
            '5',
            '200']]
)

parser = PandasDataFrameOutputParser(dataframe=df)
print(parser.get_format_instructions())

In [ ]:
nutri_prompt_template = ("You are a search assistant. The human user will give you a description of their meals, portion size and the time they consumed it. Your task is to analyse the meals, log the date and time of each meal,"
    "and then decompose the meal and its ingredients into their nutrient content. Use google search tool to get nutrient data from https://www.fatsecret.com/calories-nutrition/ if possible. MAke sensible guesses about portions where user does not provide"
    "Output the amount of carbohydrates, fats, proteins in grams, and calories. "
    "Please give a unique id to each meal, that is repeated across its components."
    "Answer with the nutritional breakdown of the meal in a json format (and nothing else) as "
    "[\{{\'Column1': 'Value1', 'Column2': 'Value2', 'Column3': 'Value3'\}},"
    "\{{\'Column1': 'Value4', 'Column2': 'Value5', 'Column3': 'Value6'\}}]"
    "The following are the key/column names, and you must extract the value from the meal analysis"
     "- meal_id"
    "- date"
    "- time"
    "- meal_component"
    "- quantity_per_serving"
    "- carbohydrates"
    "- protein"
    "- fat"
    "- calories"
    "Only answer with the json key-value list, and nothing else.")

In [ ]:
parser = JsonOutputParser()
parser.get_format_instructions()

In [ ]:
# Set up the prompt.
prompt_template = PromptTemplate(
    template = nutri_prompt_template + "{format_instructions}" + "\n{meal_description}\n",
    input_variables=["meal_description"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
meal_description = ("today is 24th feb, I ate a cheeseburger with some slaw for lunch, and then a plain dosa for dinner, and yesterday I ate chowmein at 5pm")

# Generate the prompt
filled_prompt = prompt_template.invoke({"meal_description": meal_description})
filled_prompt

### access GenAI chat API

In [ ]:
class MODEL_ID(LowercaseStrEnum):
    GEMINI_1_5_FLASH_8B = "gemini-1.5-flash-8b"
    GEMINI_2_0_FLASH = "gemini-2.0-flash"
    GEMINI_1_5_FLASH = "gemini-1.5-flash"


In [ ]:

# Define custom GenerateContentConfig parameters
client_options = {
    "temperature": 0.0,  # Controls randomness
    "max_output_tokens": 1000,  # Limit on output tokens
    "stop_sequences": ["\n"],   # Optional stop sequences
}


# Initialize ChatGoogleGenerativeAI with client options
llm = ChatGoogleGenerativeAI(
    model = MODEL_ID.GEMINI_1_5_FLASH,
    google_api_key = GOOGLE_API_KEY,
    max_output_tokens = 1000,
    temperature=0,
    top_p=0.95,
    top_k=20,
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    ))
   #client_options =  {'generate_content_config': {'max_output_tokens': 100}})

# Invoke the model with a prompt
response = llm.invoke(filled_prompt.text )
print(response.content)


In [ ]:
response

In [ ]:
parsed_json = parser.parse(response.content)


In [ ]:
df = pd.DataFrame.from_records(parsed_json)
df.groupby('date').cumcount() + 1
df

### try google gemini directly

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

google_search_tool = Tool(
    google_search = GoogleSearch()
)

response_direct = client.models.generate_content(
    model=MODEL_ID.GEMINI_2_0_FLASH,
    contents=filled_prompt, 
    config=types.GenerateContentConfig(
        tools=[google_search_tool],
         response_modalities=["TEXT"],
    )
)


In [ ]:
print(response_direct_text)

## agent

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Combine LLM and tools into an agent
tools = [search_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Example query that requires search
query = "What are the latest advancements in AI research? Please answer in not more than 5 lines"

response = agent.run(query)

print(response)


### postgres

In [ ]:
f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
df

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection details
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'postgres'

# Create SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Create a DataFrame
#data = {'name': ['Alice', 'Bob', 'Charlie'], 'age': [25, 30, 35]}
#df = pd.DataFrame(data)

# Store DataFrame into PostgreSQL
df.to_sql('table_name', engine, index=False, if_exists='replace')


In [ ]:
r		

In [ ]:
df.to_sql('table_name', engine, index=False, if_exists='append')


In [ ]:
df

In [ ]:
df['protein'].astype(int)

In [ ]:
json_records = df.to_dict("records")

In [ ]:
from st_supabase_connection import SupabaseConnection
from src.config import PostgresConfig, DEFAULT_POSTGRES_CONFIG_LOCAL_v1
from src.data_utils import COLUMN_NAMES
import streamlit as st
from supabase import create_client, Client

POSTGRES_TABLENAME = "nutrition_data"
POSTGRES_HOST_LOCAL = "local"
POSTGRES_HOST_SUPABASE = "supabase"
ALLOWED_POSTGRES_HOSTS = [POSTGRES_HOST_SUPABASE, POSTGRES_HOST_LOCAL]


## app level config
STREAMLIT_APP_POSTGRESS_HOST = POSTGRES_HOST_SUPABASE
STREAMLIT_APP_POSTGRES_CONFIG = DEFAULT_POSTGRES_CONFIG_LOCAL_v1

# Initialize connection.
# Uses st.cache_resource to only run once.
#@st.cache_resource
def init_connection():
    url = st.secrets["SUPABASE_URL"]
    key = st.secrets["SUPABASE_KEY"]
    return create_client(url, key)

supabase = init_connection()


In [ ]:
# Perform the query
response = supabase.table("nutrition_data").select("*").execute()

# Check if the response contains data
if response and response.data:
    for row in response.data:
        print(row)
else:
    print("No data retrieved or query failed.")


In [ ]:
# Perform the query
table_name = "nutrition_data"
response = supabase.table(table_name).select("*").execute()

# Check if the response contains data
if response and response.data:
    for row in response.data:
        print(row)
else:
    print("No data retrieved or query failed.")


In [ ]:
response

In [ ]:
table_name = "nutrition_data"
from typing import List, Dict
def insert_data_in_supabase(json_records: List[Dict],
                            table_name = table_name,
                            conn = supabase_conn):
    try:
        response = (
            conn.table(table_name)
            .insert(json_records)
            .execute()
        )
        return response
    except Exception as exception:
        return exception


In [ ]:
insert_data_in_supabase(json_records=json_records,
                                    table_name=table_name,
                                    conn=supabase_conn)

In [ ]:
import secrets
print(secrets.token_urlsafe(32))
